In [45]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m4_survey_data.sqlite
!mv m4_survey_data.sqlite ./files

--2021-02-19 19:53:32--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m4_survey_data.sqlite
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36679680 (35M) [application/octet-stream]
Saving to: ‘m4_survey_data.sqlite’

m4_survey_data.sqli 100%[===================>]  34.98M  12.5MB/s    in 2.8s    

2021-02-19 19:53:35 (12.5 MB/s) - ‘m4_survey_data.sqlite’ saved [36679680/36679680]



In [46]:
from glob import glob
import pandas as pd
import sqlite3
import os

In [47]:
path_SQL = r'./files/*.sqlite'
sql = glob(os.path.join(path_SQL))[0]

In [48]:
conn = sqlite3.connect(sql)

In [49]:
query = '''
    select name as Table_Name from sqlite_master where
    type = 'table';
'''

tables = pd.read_sql_query(query,conn)
tables

,Table_Name
0,EduOther
1,DevType
2,LastInt
3,JobFactors
4,WorkPlan
5,WorkChallenge
6,LanguageWorkedWith
7,LanguageDesireNextYear
8,DatabaseWorkedWith
9,DatabaseDesireNextYear


In [50]:
conn.close()